In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

# np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# pandas version
print('pandas',pd.__version__)

In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')\

train = pd.read_csv('./drive/My Drive/DACON/data/train.csv')
train_copy = train.copy()

In [ ]:
# resize the input dataframe of (28,28) images to array of (1,112,112,1) images
def resize(input_imgs) :

    # zero padding
    img = np.zeros((input_imgs.shape[0],112,112,1))
    npad = ((42,42),(42,42))
    for i in range(img.shape[0]):
        tmp = np.divide(input_imgs.iloc[i].values,255)
        img[i] = np.pad(tmp.reshape(28,28),npad,'constant',constant_values=(0)).reshape(1,112,112,1) 
    
    return img

In [ ]:
def avg_digit(input_imgs,th):
    
    img = np.zeros((10,784))

    for i in range(10):
        digit_data = input_imgs[input_imgs['digit']==i]
        digit = digit_data.iloc[0,3:]
        for j in range(1,digit_data.shape[0]-1):
            digit += digit_data.iloc[j,3:].values 
        digit = np.divide(digit,digit_data.shape[0])
        digit[digit< th]=0
        img[i] = digit

    return img

In [ ]:
a = avg_digit(train_copy)
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(1,10,1+i)
    plt.imshow(a[i].reshape(28,28),cmap='gray')
    plt.axis('off')

In [ ]:
def avg_letter(input_imgs,th):

    img = np.zeros((26,784))

    for i in range(26):
        letter_data = input_imgs[input_imgs['letter']==chr(i+65)]
        letter = letter_data.iloc[0,3:]
        for j in range(1,letter_data.shape[0]-1):
            letter += letter_data.iloc[j,3:].values 
        letter = np.divide(letter,letter_data.shape[0])
        letter[letter< th ]=0
        img[i] = letter

    return img

In [ ]:
a = avg_letter(train_copy)
plt.figure(figsize=(20,20))
for i in range(26):
    plt.subplot(1,26,i+1)
    plt.imshow(a[i].reshape(28,28),cmap='gray')
    plt.axis('off')

In [ ]:
# create the image label for u-net
def img_label(input_imgs) :

    label_array = np.zeros((10,26,784))

    # try1. avg of images with same digit and letter
    """for i in range(10): # for each digit
        for j in range(26): # for each letter
            label_set = input_imgs[(input_imgs['digit']==i) & (input_imgs['letter']==chr(j+65))]
            label = label_set.iloc[0,3:]
            for k in range(1,label_set.shape[0]-1):
                label += label_set.iloc[k,3:] 
            # normalize
            label = np.divide(label,label_set.shape[0])
            label[label < 35]=0
            label_array[i][j] = label"""

    # trsy2. avg of digit + avg of letter
    """digit = avg_digit(input_imgs)
    letter = avg_letter(input_imgs)

    for i in range(10): # for each digit
        for j in range(26): # for each letter
            label_array[i][j] = np.divide( (digit[i]+letter[j]) ,2)
    """

    """# try3. overlap avg of digit and letter
    digit = avg_digit(input_imgs,80)
    letter = avg_letter(input_imgs,40)

    for i in range(10): # for each digit
        for j in range(26): # for each letter
            ov = train_copy.copy().iloc[0,3:]
            for k in range(784):
                if letter[j][k]==0 : label_array[i][j][k] = letter[j][k]
                else : label_array[i][j][k] = min((digit[i][k]+letter[j][k]),255)"""

    # try4. digit only survive
    digit = avg_digit(input_imgs,80)
    letter = avg_letter(input_imgs,40)

    for i in range(10): # for each digit
        for j in range(26): # for each letter
            ov = train_copy.copy().iloc[0,3:]
            for k in range(784):
                if (letter[j][k]>0) & (digit[i][k]>0) : label_array[i][j][k] = min((digit[i][k]+letter[j][k]),255)
                else : label_array[i][j][k] = 0

    return label_array

In [ ]:
a = img_label(train_copy)
plt.imshow(a[2][2].reshape(28,28),cmap='gray')

In [ ]:
# prepare data for unet
x_train = resize(train_copy.iloc[:,3:])

y_label = img_label(train_copy) 
y_28 = train_copy.iloc[:,3:].copy()

for i in range(train_copy.shape[0]):
    y_28.iloc[i,:] = y_label[ train_copy.iloc[i,1] ][ ord(train_copy.iloc[i,2])-65 ] 

y_train = resize( y_28 )

plt.subplot(1,2,2)
plt.imshow(y_train[0].reshape(112,112),cmap='gray')

# split training and validation set
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=15)

In [ ]:
# unet architecture

def unet(input_size = (112,112,1)):
    inputs = Input(input_size)
    N = 16
    conv1 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(16*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(16*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(8*N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(8*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(4*N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(4*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(2*N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(2*N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(N, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(N, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = "binary_crossentropy", metrics = ['accuracy'])
    
    # model.summary()

    # if(pretrained_weights): model.load_weights(pretrained_weights)

    return model

In [ ]:
# with data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

# set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

# fit the model

epochs = 5
batch_size = 50

model_u = unet()
history = model_u.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, 
                              validation_data = (x_val,y_val),
                              steps_per_epoch=x_train.shape[0]// batch_size
                              ,callbacks=[learning_rate_reduction]
                             )

# plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# prepare data for CNN
x_train_ = resize(train_copy.iloc[:,3:])

for i in range(x_train_.shape[0]):
    x_train_[i] = model_u.predict(x_train_[i].reshape(1,112,112,1))
    if i%100==0 : print(i)

# label encoding
y_train_ = train_copy['digit'].values
y_train_ = to_categorical(y_train_,num_classes = 10)

# split training and validation set
x_train_, x_val_, y_train_, y_val_ = train_test_split(x_train_,y_train_,test_size=0.1,random_state=15)

In [ ]:
plt.figure(figsize=(15,15))
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(x_train_[i].reshape(112,112),cmap='gray')
    plt.title('digit:{}'.format(np.where(y_train_[i]==1)[0][0]),fontsize=20)

plt.figure(figsize=(15,15))
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(x_train_[i+3].reshape(112,112),cmap='gray')
    plt.title('digit:{}'.format(np.where(y_train_[i+3]==1)[0][0]),fontsize=20)

In [ ]:
# set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (112,112,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

# define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
# with data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train_)

In [ ]:
epochs = 10
batch_size = 50

# fit the model
history = model.fit_generator(datagen.flow(x_train_,y_train_, batch_size=batch_size), epochs = epochs, validation_data = (x_val_,y_val_),
                              steps_per_epoch=x_train.shape[0]// batch_size, callbacks=[learning_rate_reduction])

# plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# Predict the values from the validation dataset
y_pred = model.predict(x_val_)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 
# Convert validation observations to one hot vectors
y_true = np.argmax(y_val_,axis = 1) 


# Errors are difference between predicted labels and true labels
errors = (y_pred_classes - y_true != 0)

y_pred_classes_errors = y_pred_classes[errors]
y_pred_errors = y_pred[errors]
y_true_errors = y_true[errors]
x_val_errors = x_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((112,112)),cmap='gray')
            ax[row,col].axis('off')
            ax[row,col].set_title("Predicted:{} / True:{}".format(pred_errors[error],obs_errors[error]),fontsize=10)
            n += 1

# Probabilities of the wrong predicted numbers
y_pred_errors_prob = np.max(y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(y_pred_errors, y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, x_val_errors, y_pred_classes_errors, y_true_errors)

In [ ]:

"""history = model.fit(x_train_, y_train_, 
                    batch_size = batch_size, epochs = epochs, 
                    validation_data = (x_val_, y_val_))

# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)"""
